# Class: LLM Fundamentals - Class Lab 1b - OpenAI LLM API
# Topic: How an LLM generates the next token?

In [1]:
# OPENAI_API_KEY

In [2]:
print("Hello, LLM Fundamentals class ....")

Hello, LLM Fundamentals class ....


In [3]:
import math
import os, requests

# Sample code to show converting raw numbers to log and inverse-log

In [4]:
import math

original_prob = 0.7
logprob = math.log(original_prob)
prob = math.exp(logprob)

print(f"Original probability: {original_prob}")
print(f"Log probability: {logprob}")
print(f"Reconstructed probability: {prob}")

Original probability: 0.7
Log probability: -0.35667494393873245
Reconstructed probability: 0.7


# Create your own OpenAI API Key

In [5]:
from google.colab import userdata
# userdata.get('OPENAI_API_KEY')



---



In [6]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
text_to_embed = "How are you?"
# text_to_embed = "The quick brown fox jumps over the"
# text_to_embed = "The dog sat on"
# text_to_embed = "I have a dream"

response = client.embeddings.create(
    input=text_to_embed,
    model="text-embedding-3-small"
)

embedding_vector = response.data[0].embedding

print(f"Embedding for the text: {embedding_vector}")
print(f"Length of the embedding vector: {len(embedding_vector)}")

Embedding for the text: [0.02755492739379406, -0.02655552513897419, -0.027055226266384125, 0.057774923741817474, -0.0035187259782105684, -0.002834612037986517, 0.0318380743265152, 0.010148684494197369, -0.03412242233753204, -0.04466372728347778, -0.03450314700603485, -0.009422928094863892, -0.02228427305817604, -0.0004093530587852001, -0.013206376694142818, 0.021320564672350883, -0.028530532494187355, 0.014741172082722187, 0.0079595185816288, 0.0040570939891040325, 0.025794075801968575, -0.019155194982886314, 0.0015407439786940813, 0.02276017889380455, 0.050493571907281876, -0.0005216370336711407, 0.006888731848448515, 0.0466863252222538, -0.010844696313142776, -0.023723887279629707, 0.04266492649912834, -0.03971431031823158, 0.02534196712076664, 0.035431161522865295, -0.007108837831765413, 0.04045196622610092, -0.05715624615550041, 0.02054721862077713, -0.004205814562737942, -0.06339061260223389, -0.002703737933188677, -0.047495365142822266, 0.010553203523159027, 0.01959540694952011, 

# "text-embedding-3-small" has default dimensions=1526, it can be changed to 512. By using the dimensions parameter, you can control the trade-off between embedding size and the model's ability to capture the nuances of the text.

# Embeddings are numerical representations (vectors) of text that capture its semantic meaning. To get them, you will make an API call to the embeddings endpoint, which returns a list of floating-point numbers representing the input text.

# OpenAI API call for text completions to generate the "next token" using gpt-4o

Reference:
https://platform.openai.com/docs/api-reference/chat


logprobs: Whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message.


top_logprobs: An integer between 0 and 20 specifying the number of most likely tokens to return at each token position, each with an associated log probability.

In [7]:
# api_key = os.environ.get("OPENAI_API_KEY")
api_key = userdata.get('OPENAI_API_KEY')

In [8]:
# print(api_key)

# Using gpt-4o to generate text completion response

# Notice the usage of "logprobs" and "top_logprobs"

https://developers.openai.com/cookbook/examples/using_logprobs

In [9]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers={
        "Authorization":  f"Bearer {api_key}",
        "Content-Type": "application/json",
    },
    json={
        "model": "gpt-4o",
        "messages": [
            {"role": "user", "content": "The capital of France is"}
        ],
        "logprobs": True,
        "top_logprobs": 5
    }
)

# A successful execution of response should result in response of <Response [200]>

In [10]:
print(response)

<Response [200]>


# Convert the response to json, then extract words and probabilities

In [11]:
response_json = response.json()
logprobs = response_json["choices"][0]["logprobs"]
next_token_logprobs = logprobs["content"][0]["top_logprobs"]

# Check the contents of the dictionary

bytes are "vector representation" of the words and symbols

In [12]:
next_token_logprobs

[{'token': 'Paris',
  'logprob': -0.004078878089785576,
  'bytes': [80, 97, 114, 105, 115]},
 {'token': 'The', 'logprob': -5.5040788650512695, 'bytes': [84, 104, 101]},
 {'token': 'the', 'logprob': -15.25407886505127, 'bytes': [116, 104, 101]},
 {'token': ' Paris',
  'logprob': -15.37907886505127,
  'bytes': [32, 80, 97, 114, 105, 115]},
 {'token': 'Par', 'logprob': -18.879079818725586, 'bytes': [80, 97, 114]}]

# Take inverse-log to convert probabilities to normal numbers

In [13]:
token_prob = {}
for item in next_token_logprobs:
    token, logprob = item["token"], item["logprob"]
    prob = math.exp(logprob)
    print(token, prob)
    token_prob[token] = prob

Paris 0.9959294292347572
The 0.004070135999190959
the 2.372672094117407e-07
 Paris 2.0938757739075173e-07
Par 6.322950928300138e-09


In [14]:
token_prob

{'Paris': 0.9959294292347572,
 'The': 0.004070135999190959,
 'the': 2.372672094117407e-07,
 ' Paris': 2.0938757739075173e-07,
 'Par': 6.322950928300138e-09}

# Apply "Greedy Sampling" (token with the highest probability to be the "Next Token")

In [15]:
# Find the key with the highest value
highest_value_key = max(token_prob, key=token_prob.get)

In [16]:
print(f"The key with the highest value is: {highest_value_key}")

The key with the highest value is: Paris


# 1) Remember GPU Savings: Runtime >> Disconnect and delete run time.
# 2) Check you do not have any active sessions on the VM.

In [17]:
from openai import OpenAI
from math import exp
import numpy as np
from IPython.display import display, HTML
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [18]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [19]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Return only the name of the category, and nothing else.
MAKE SURE your output is one of the four categories stated.
Article headline: {headline}"""

In [20]:
headlines = [
    "Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.",
    "Local Mayor Launches Initiative to Enhance Urban Public Transport.",
    "Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut",
]

In [21]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4o",
    )
    print(f"Category: {API_RESPONSE.choices[0].message.content}\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your Op*******************************var>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.
Category: Technology


Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.
Category: Politics


Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut
Category: Art

In [ ]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4o-mini",
        logprobs=True,
        top_logprobs=2,
    )
    top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
    html_content = ""
    for i, logprob in enumerate(top_two_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )
    display(HTML(html_content))
    print("\n")